# Run a test

Start exploring the metrics and threshold tests that ValidMind offers out-of-the-box by running one of the tests and logging the results to your model documentation.

What are metrics or tests?

- **Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

- **Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

This interactive notebook provides a step-by-step guide for loading a demo dataset, preprocessing the raw data, training a model for testing, settting up test inputs, initializing the required ValidMind objects, running the test, and then logging the results to ValidMind.

## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).


## Install the client library


In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **Binary classification** as the template and **Marketing/Sales - Attrition/Churn Management** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the demo dataset


In [ ]:
from validmind.datasets.classification import customer_churn as demo_dataset

# You can also import customer_churn like this:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### Prepocess the raw dataset


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## Train a model for testing

We train a simple customer churn model for our test.


In [ ]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## Set up test inputs and run the test

### Initialize ValidMind objects


In [ ]:
vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column,
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset",
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column,
)

vm_model = vm.init_model(model, input_id="model")

### Assign predictions to the datasets

We can now use the `assign_predictions()` method from the `Dataset` object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:


In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
)
vm_test_ds.assign_predictions(
    model=vm_model,
)

### Run the test

Individual tests can be easily run by calling the `run_test` function provided by the `validmind.tests` module. The function takes the following arguments:

- `test_id`: The ID of the test to run. To find a particular test and get its, refer to the `explore_tests.ipynb` notebook
- `params`: A dictionary of parameters for the test. These will override any `default_params` set in the test definition. Refer to the `explore_tests.ipynb` notebook to find the default parameters for a test.

You can then pass in any inputs for the test as keyword arguments. Most likely, these will be `dataset` and `model` objects. Again, you may refer to the `explore_tests.ipynb` notebook to find the required inputs for a test.


In [ ]:
test = vm.tests.run_test(
    test_id="validmind.model_validation.sklearn.TrainingTestDegradation",
    params={},  # can be used to set overrides to the test's default parameters
    inputs={"model": vm_model, "datasets": (vm_train_ds, vm_test_ds)},
)

### Log the test results to ValidMind

After the test has been run, you can save the results to ValidMind by calling the `log` method of the test object returned after running the test:


In [ ]:
test.log()

## Next steps

You can look at the results of this test plan right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation in the ValidMind Platform UI:

1. In the [Platform UI](https://app.prod.validmind.ai), go to the **Documentation** page for the model you registered earlier.

2. Expand **Model Development**

What you can see now is a more easily consumable version of the model diagnosis tests you just performed, along with other parts of your model documentation that still need to be completed.

If you want to learn more about where you are in the model documentation process, take a look at [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html).

### Find your next how-to guide

As always, to learn more about ValidMind, please visit our [documentation](https://docs.validmind.ai/).